In [3]:
import psycopg2 as pg2
import pandas as pd

In [52]:
def createDB():
    conn = pg2.connect(
        host = 'localhost',
        dbname = 'mydb',
        user = 'postgres',
        password = 'hsgiri'
    )
    conn.autocommit = True
    cur = conn.cursor()
    return cur, conn

In [54]:
choco_sales_csv = pd.read_csv('/Users/harishankargiri/Downloads/chocolate_sales.csv')
choco_sales_csv.head() # to print the first five rows of a dataset

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,05-Jan-22,"$6,596",104
1,Emma Clarkson,USA,Dark Choco,04-Dec-23,"$8,610",97
2,Liam Peterson,Canada,Milk Choco,05-Dec-22,"$9,850",194
3,Olivia Barnes,Germany,White Choco,21-May-22,"$7,772",256
4,Ethan Wright,France,Caramel Choco,07-Feb-22,"$4,852",205


In [56]:
choco_supl_csv = pd.read_csv('/Users/harishankargiri/Downloads/chocolate_suppliers.csv')
choco_supl_csv.head()
choco_supl_csv = choco_supl_csv.drop('Last Shipment Date', axis = 1) # Remove a col from dataframe by name, 1:col, 0:row

In [58]:
choco_supl_csv.head()

,Supplier Name,Country,Product Supplied,Supply Quantity (kg)
0,Cocoa Delights,Ghana,Cocoa Beans,807
1,Sweet Beans Ltd.,Brazil,Sugar,1295
2,Choco Source,Ivory Coast,Milk Powder,1932
3,Premium Cocoa,Ecuador,Vanilla Extract,2905
4,Gourmet Ingredients,Indonesia,Cocoa Butter,4185


In [60]:
choco_inve_csv = pd.read_csv('/Users/harishankargiri/Downloads/chocolate_inventory.csv')
choco_inve_csv.head()

,Warehouse,Product,Stock Level,Reorder Level,Last Restocked
0,London,Mint Chip Choco,376,116,01-Nov-23
1,New York,Dark Choco,901,176,27-Oct-22
2,Toronto,Milk Choco,121,172,01-Apr-22
3,Berlin,White Choco,246,95,16-Dec-23
4,Paris,Caramel Choco,349,146,31-Jan-23


In [62]:
cur, conn = createDB()

In [64]:
create_choco_sales = cur.execute("""
    CREATE TABLE IF NOT EXISTS choco_sales (
        id SERIAL PRIMARY KEY,
        sales_person VARCHAR(100),
        country VARCHAR(50),
        product VARCHAR(100),
        sale_date DATE,
        amount NUMERIC(10,2),
        boxes_shipped INT
    )
""")

create_choco_supl = cur.execute("""
    CREATE TABLE IF NOT EXISTS choco_supl (
        id SERIAL PRIMARY KEY,
        supplier_name VARCHAR(100),
        country VARCHAR(50),
        product_supplied VARCHAR(100),
        supply_quantity INT
    )
""")

create_choco_inve = cur.execute("""
    CREATE TABLE if not exists choco_inve (
        id SERIAL PRIMARY KEY,
        warehouse VARCHAR(100),
        product VARCHAR(100),
        stock_level INT,
        reorder_level INT,
        last_restocked DATE
    )
""")


In [110]:
insert_choco_sales = ("""
    INSERT INTO choco_sales (
        sales_person, country, product, sale_date, amount, boxes_shipped
    ) 
    VALUES (%s, %s, %s, %s, %s, %s)
""")

insert_choco_supl = ("""
    INSERT INTO choco_supl (
        supplier_name, country, product_supplied, supply_quantity
    ) 
    VALUES (%s, %s, %s, %s)
""")

insert_choco_inve = ("""
    insert into choco_inve (
        warehouse, product, stock_level, reorder_level, last_restocked
    )
    values (%s, %s, %s, %s, %s)
""")

In [112]:
for i, row in choco_sales_csv.iterrows():
    row_list = list(row)
    row_list[4] = float(row_list[4].replace("$", "").replace(",", ""))
    #print(i, list(row))
    cur.execute(insert_choco_sales, row_list)

In [114]:
for i, row in choco_supl_csv.iterrows():
    cur.execute(insert_choco_supl, list(row))

In [118]:
for i, row in choco_inve_csv.iterrows():
    cur.execute(insert_choco_inve, list(row))